In [15]:
from scrapfly import ScrapflyClient, ScrapeConfig
from bs4 import BeautifulSoup
import pandas as pd
import time

# tighten your timeouts & disable retries
client = ScrapflyClient(
    key="scp-live-7a100878a690435eb6a2ab5d9d5fbb4b",
    connect_timeout=10,
    web_scraping_api_read_timeout=30
)

def fetch_infinite(base_url):
    cfg = ScrapeConfig(
        url=base_url,
        render_js=True,
        asp=True,
        country="US",
        proxy_pool=ScrapeConfig.PUBLIC_RESIDENTIAL_POOL,
        auto_scroll=True,                         # let Scrapfly scroll until no more content
        wait_for_selector="div.job_seen_beacon:nth-of-type(30)",
        retry=False
    )
    res = client.scrape(cfg)                     # ← positional!
    html = res.result.get("content", "")         # or res.scrape_result["content"]
    return BeautifulSoup(html, "html.parser")

def fetch_detail_page(url):
    cfg = ScrapeConfig(
        url=url,
        render_js=False,
        asp=True,
        country="US",
        proxy_pool=ScrapeConfig.PUBLIC_RESIDENTIAL_POOL,
        retry=False
    )
    res = client.scrape(cfg)                     # ← positional!
    html = res.result.get("content", "")         # or res.scrape_result["content"]
    return BeautifulSoup(html, "html.parser")


def parse_listings(soup):
    cards = soup.select("div.job_seen_beacon")
    data = []
    for card in cards:
        # required elements
        title_el = card.select_one("h2.jobTitle span")
        link_el  = card.select_one("a")
        if not title_el or not link_el:
            # skip cards missing the basics
            continue

        link = link_el["href"]
        if not link.startswith("http"):
            link = "https://www.indeed.com" + link

        # optional elements—check for None before calling get_text
        company_el = card.select_one(".companyName")
        loc_el     = card.select_one(".companyLocation")
        summary_el = card.select_one(".job-snippet")

        data.append({
            "title":    title_el.get_text(strip=True),
            "company":  company_el.get_text(strip=True) if company_el else "",
            "location": loc_el.get_text(strip=True) if loc_el else "",
            "summary":  " ".join(summary_el.stripped_strings) if summary_el else "",
            "link":     link
        })
    return data

# 3) Scrape multiple pages until you have 30
base = "https://www.indeed.com/jobs?q=nurse&l=United+States"
jobs = []
page = 0

# 1) Grab listings
print("Fetching infinite-scroll page…")
soup = fetch_infinite(base)

cards = soup.select("div.job_seen_beacon")
print("Found", len(cards), "cards")

""" # Parse out *all* the cards, then slice the first 30
all_cards = soup.select("div.job_seen_beacon")
print("Found", len(all_cards), "cards in total.")
jobs = parse_listings(soup)[:30]   # reuse your parse_listings()

# 2) Grab descriptions
for i, job in enumerate(jobs[:30], 1):
    print(f"Fetching detail {i}/30:", job["link"])
    soup = fetch_detail_page(job["link"])
    desc = soup.select_one("#jobDescriptionText")
    job["description"] = desc.get_text("\n", strip=True) if desc else ""

# 5) Save to CSV
df = pd.DataFrame(jobs[:30])
df.to_csv("nurse_jobs.csv", index=False)
print("Done, scraped", len(df), "jobs. Sample:")
df.head() """

Fetching infinite-scroll page…
Found 0 cards


' # Parse out *all* the cards, then slice the first 30\nall_cards = soup.select("div.job_seen_beacon")\nprint("Found", len(all_cards), "cards in total.")\njobs = parse_listings(soup)[:30]   # reuse your parse_listings()\n\n# 2) Grab descriptions\nfor i, job in enumerate(jobs[:30], 1):\n    print(f"Fetching detail {i}/30:", job["link"])\n    soup = fetch_detail_page(job["link"])\n    desc = soup.select_one("#jobDescriptionText")\n    job["description"] = desc.get_text("\n", strip=True) if desc else ""\n\n# 5) Save to CSV\ndf = pd.DataFrame(jobs[:30])\ndf.to_csv("nurse_jobs.csv", index=False)\nprint("Done, scraped", len(df), "jobs. Sample:")\ndf.head() '